Copyright (c) 2025 Christian Oechler

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE. 

## Importing the necessary python packages

In [ ]:
import json
from pathlib import Path
import shutil
from tqdm.notebook import tqdm
import hashlib as hash

## Defination of the helper functions

In [42]:
def get_sha256_checksum(filepath: str) -> str:
    BLOCKSIZE = 65536
    
    sha = hash.sha256()
    
    with open(filepath, 'rb') as handler:
        file_buffer = handler.read(BLOCKSIZE)
            
        while len(file_buffer) > 0:
            sha.update(file_buffer)
            file_buffer = handler.read(BLOCKSIZE)
            
    return sha.hexdigest()

In [20]:
def create_filename(metadata) -> str:
    return f'{metadata["video_time"]}'

In [ ]:
def create_path(metadata) -> Path:
    video_date = metadata["video_date"]
    video_camera = metadata["video_camera"]

    video_date_elements = video_date.split("-")
    day = video_date_elements[0]
    month = video_date_elements[1]
    year = video_date_elements[2]

    return Path(f'{video_camera}/{year}/{month}/{day}')

In [62]:
def copy_videofiles(source_file_path, destination_file_path, attempts=5) -> bool:
    for _ in range(attempts):
        shutil.copy2(source_file_path, destination_file_path)

        # Calculates the checksum of the source and destination file
        checksum_source_file = get_sha256_checksum(f'{source_file_path}')
        checksum_destination_file = get_sha256_checksum(f'{destination_file_path}')

        # Checks that the file was successfully copied
        if checksum_source_file == checksum_destination_file:
            return True

    return False

## Extraction of the metadata from the video file content

### Setup the system

In [60]:
# Defines the base directory of the video files
data_directory = "data"
original_video_diretory = "videos"
renamed_video_directory = "renamed_videos"

In [63]:
original_video_diretory = Path("original_video_diretory")
renamed_video_directory = Path("renamed_videos")

with open(f'{data_directory}/data.json', 'r') as json_file:
    video_list = json.load(json_file)

with tqdm(video_list.keys(), desc="Fortschritt", unit="Datei") as pbar:
    for i, key in enumerate(pbar):
        video_metadata = video_list[key]
        source_file_path = Path(str(key))

        # Checks the existance of the source video file
        if not source_file_path.exists() or not source_file_path.is_file():
            print(f'Die Videodatei {source_file_path} existiert nicht.')
            continue
        
        # Creates the nessessary directories
        path = renamed_video_directory / create_path(video_metadata)
        path.mkdir(parents=True, exist_ok=True)
    
        # Copy the file to the new directory
        filename = Path(f'{create_filename(video_metadata)}{source_file_path.suffix}')
        destination_file_path = path / filename

        # Skips further processing, if destination video file already exists
        if destination_file_path.exists():
            continue

        pbar.set_postfix_str(f'{source_file_path.name} => {destination_file_path}')
        
        result = copy_videofiles(source_file_path, destination_file_path)

        if not result:
            print(f'Es ist ein Fehler beim Kopieren der Datei "{key}" aufgetreten.')

Fortschritt:   0%|          | 0/1144 [00:00<?, ?Datei/s]